In [245]:
import twitter
import tweepy

In [246]:
auth = tweepy.OAuthHandler('wVOV5ZIj1VjkT7vteqlBQpqhW', '3cbZt70binbwKjNpaJCYAljgnyafWO4v87T7m3KvSBG7JXgrks')
auth.set_access_token('273450258-rWsQEagmh6atowoapPLXJYGEIAIKKxMLOnt683OV','fsDGkgLVzJyMJAHa0968ojP24O3XhhfgvXvAHixD5IqdM')
api = tweepy.API(auth)

In [247]:
alltweets=[]

In [248]:
new_tweets=api.user_timeline(screen_name='ArmenianUnified',count=200,tweet_mode='extended')
alltweets.extend(new_tweets)
oldest = alltweets[-1].id - 1

In [249]:
while len(new_tweets) > 0:
        print(f"getting tweets before {oldest}")
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets=api.user_timeline(screen_name = 'ArmenianUnified',count=200,max_id=oldest,tweet_mode='extended')
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        print(f"...{len(alltweets)} tweets downloaded so far")

getting tweets before 1317352745272287231
...400 tweets downloaded so far
getting tweets before 1313464774156603393
...600 tweets downloaded so far
getting tweets before 1311198814930440191
...800 tweets downloaded so far
getting tweets before 1283633970714947583
...944 tweets downloaded so far
getting tweets before 1239809172201508864
...944 tweets downloaded so far


In [250]:
outtweets = [[tweet.id_str,
              tweet.created_at,
              tweet.full_text,
              tweet.user.screen_name,
              tweet.entities['hashtags'],
              tweet.geo,
              tweet.user.time_zone]
             for tweet in alltweets]

In [251]:
import pandas as pd
df=pd.DataFrame(outtweets,columns=['ID','Date','Text','User','Hashtags','Geo','Timezone'])
df.head()

,ID,Date,Text,User,Hashtags,Geo,Timezone
0,1322153249370234880,2020-10-30 12:27:53,Defense Army captured another terrorist the in...,ArmenianUnified,[],None,None
1,1322084752787210240,2020-10-30 07:55:42,"At the moment, intense fighting is taking plac...",ArmenianUnified,[],None,None
2,1322084656087470080,2020-10-30 07:55:19,Earlier today #Azerbaijan’i Armed Forces launc...,ArmenianUnified,"[{'text': 'Azerbaijan', 'indices': [14, 25]}]",None,None
3,1322064226547077121,2020-10-30 06:34:09,"84-year-old Misha Melkumyan, a civilian captur...",ArmenianUnified,"[{'text': 'Azerbaijan', 'indices': [52, 63]}]",None,None
4,1322057230418661378,2020-10-30 06:06:21,The forces of the adversary suffered huge loss...,ArmenianUnified,[],None,None


In [254]:
df.shape

(944, 7)

In [255]:
alltweets2=[]

In [256]:
new_tweets=api.user_timeline(screen_name='ShStepanyan',count=200,tweet_mode='extended')
alltweets2.extend(new_tweets)
oldest = alltweets2[-1].id - 1

In [257]:
while len(new_tweets) > 0:
        print(f"getting tweets before {oldest}")
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets=api.user_timeline(screen_name = 'ShStepanyan',count=200,max_id=oldest,tweet_mode='extended')
        
        #save most recent tweets
        alltweets2.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets2[-1].id - 1
        
        print(f"...{len(alltweets2)} tweets downloaded so far")

getting tweets before 1312331467775782916
...342 tweets downloaded so far
getting tweets before 1284102717200388097
...342 tweets downloaded so far


In [258]:
outtweets = [[tweet.id_str,
              tweet.created_at,
              tweet.full_text,
              tweet.user.screen_name,
              tweet.entities.get('hashtags'),
              tweet.geo,
              tweet.user.time_zone]
             for tweet in alltweets2]

In [259]:
df2=pd.DataFrame(outtweets,columns=['ID','Date','Text','User','Hashtags','Geo','Timezone'])

In [260]:
df2.shape

(342, 7)

In [261]:
df=df.append(df2)

In [262]:
df.shape

(1286, 7)

In [242]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english',max_df=0.65)
tfidf_train=tfidf_vectorizer.fit_transform(df['Text'])
tfidf_test=tfidf_vectorizer.transform(df['Text'])

In [243]:
tfidf_test[1]

<1x6299 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

In [263]:
df['Text']

0      Defense Army captured another terrorist the in...
1      At the moment, intense fighting is taking plac...
2      Earlier today #Azerbaijan’i Armed Forces launc...
3      84-year-old Misha Melkumyan, a civilian captur...
4      The forces of the adversary suffered huge loss...
5      After midnight the forces of the adversary att...
6      Overnight, Defense Army continued to conduct r...
7      The sounds of explosions in Stepanakert at the...
8      "The priority for us is the establishment of c...
9      #Azerbaijan'i Armed Forces destroy everything ...
10     Armenian Unified Infocenter presents map illus...
11     #Armenia’s Foreign Minister Zohrab Mnatsakanya...
12     “Surprisingly, the videos Azerbaijani war crim...
13     Briefing of @ArmeniaMODTeam Representative Art...
14     #Azerbaijan continues shelling Shushi &amp; ot...
15     The statement of the #Azerbaijan|i side on the...
16     Armenian Unified Infocenter shares the images ...
17     Legitimate military targ

In [266]:
# function to preprocess speech
def clean(text):
    
    import re
    
    # Removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # Removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # Removing apostrophes
    text = re.sub("'s",'',str(text))
    # Removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # Removing quotation marks
    text = re.sub('\"','',str(text))
    # Removing hashtags and @
    text = re.sub('#','',str(text))
    text = re.sub('@','',str(text))
    text = re.sub('\´','',str(text))
    # Removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    
    return text

# preprocessing speeches
df['text_clean'] = df['Text'].apply(clean)
df.head()

,ID,Date,Text,User,Hashtags,Geo,Timezone,text_clean
0,1322153249370234880,2020-10-30 12:27:53,Defense Army captured another terrorist the in...,ArmenianUnified,[],None,None,Defense Army captured another terrorist the in...
1,1322084752787210240,2020-10-30 07:55:42,"At the moment, intense fighting is taking plac...",ArmenianUnified,[],None,None,"At the moment, intense fighting is taking plac..."
2,1322084656087470080,2020-10-30 07:55:19,Earlier today #Azerbaijan’i Armed Forces launc...,ArmenianUnified,"[{'text': 'Azerbaijan', 'indices': [14, 25]}]",None,None,Earlier today Azerbaijan’i Armed Forces launch...
3,1322064226547077121,2020-10-30 06:34:09,"84-year-old Misha Melkumyan, a civilian captur...",ArmenianUnified,"[{'text': 'Azerbaijan', 'indices': [52, 63]}]",None,None,"84 year old Misha Melkumyan, a civilian captur..."
4,1322057230418661378,2020-10-30 06:06:21,The forces of the adversary suffered huge loss...,ArmenianUnified,[],None,None,The forces of the adversary suffered huge loss...


In [268]:
# split sentences
def sentences(text):
    import re
    
    # split sentences and questions
    text = re.split('[.?]', text)
    clean_sent = []
    for sent in text:
        clean_sent.append(sent)
    return clean_sent

# sentences
df['sentences'] = df['text_clean'].apply(sentences)

In [285]:
df.reset_index(drop=True,inplace=True)

In [288]:
df.drop(['index'],axis=1,inplace=True)


KeyError: "['index'] not found in axis"

In [292]:
df.drop(['Geo','Timezone'],axis=1,inplace=True)

In [349]:
# create a dataframe containing sentences
df3 = pd.DataFrame(columns=['sentences','length','date','user'])

row_list = []

for i in range(len(df)):
    for sent in df.loc[i,'sentences']:
    
        wordcount = len(sent.split())
        date=df.loc[i,'Date']
        user=df.loc[i,'User']
        hashtags=df.loc[i,'Hashtags']
        
        dict1 = {'sentence':sent,'length':wordcount,'date':date,'user':user,'hashtags':hashtags}
        row_list.append(dict1)
    
df3 = pd.DataFrame(row_list)

In [350]:
df3.

,date,hashtags,length,sentence,user
0,2020-10-30 12:27:53,[],32,Defense Army captured another terrorist the in...,ArmenianUnified
1,2020-10-30 12:27:53,[],14,He was born in Syria’s Hama town Watch the vi...,ArmenianUnified
2,2020-10-30 12:27:53,[],1,co/aB59tMF3cG,ArmenianUnified
3,2020-10-30 07:55:42,[],12,"At the moment, intense fighting is taking plac...",ArmenianUnified
4,2020-10-30 07:55:19,"[{'text': 'Azerbaijan', 'indices': [14, 25]}]",23,Earlier today Azerbaijan’i Armed Forces launch...,ArmenianUnified


In [316]:
import spacy
import en_core_web_sm
from spacy.matcher import Matcher 
from spacy import displacy 
import visualise_spacy_tree
from IPython.display import Image, display

# load english language model
nlp = en_core_web_sm.load(disable=['ner','textcat'])

In [431]:
def prog_sent(text):
    import re
    
    patterns = [r'\b(?i)'+'smerch'+r'\b',
               r'\b(?i)'+'missile'+r'\b',
               r'\b(?i)'+'UAV'+r'\b',
               r'\b(?i)fire[a-z]+\b',
               r'\b(?i)launch[a-z]+\b',
               r'\b(?i)Stepanakert\b',
               r'\b(?i)shot[a-z]+\b',
               r'\b(?i)Shushi\b',
               r'\b(?i)attack[a-z]+\b',
               r'\b(?i)shell[a-z]+\b']

    output = []
    flag = 0
    for pat in patterns:
        if re.search(pat, text) != None:
            flag = 1
            break
    return flag 

# apply function
df3['Check_Schemes'] = df3['sentence'].apply(prog_sent)

In [432]:
attacks=df3.loc[df3['Check_Schemes']==1]

In [433]:
attacks.head()

,date,hashtags,length,sentence,user,Check_Schemes
4,2020-10-30 07:55:19,"[{'text': 'Azerbaijan', 'indices': [14, 25]}]",23,Earlier today Azerbaijan’i Armed Forces launch...,ArmenianUnified,1
13,2020-10-29 21:13:39,"[{'text': 'Azerbaijan', 'indices': [192, 203]}]",43,The sounds of explosions in Stepanakert at the...,ArmenianUnified,1
24,2020-10-29 12:34:58,"[{'text': 'Azerbaijan', 'indices': [0, 11]}, {...",14,Azerbaijan continues shelling Shushi &amp; oth...,ArmenianUnified,1
27,2020-10-29 10:44:46,"[{'text': 'Artsakh', 'indices': [167, 175]}, {...",25,Armenian Unified Infocenter shares the images ...,ArmenianUnified,1
41,2020-10-29 08:05:51,"[{'text': 'Azerbaijan', 'indices': [4, 15]}]",19,The Azerbaijan|i forces have struck Stepanaker...,ArmenianUnified,1


In [434]:
attacks.shape

(361, 6)

In [445]:
import re
attack_time=[]
time_regex=re.compile(r'\d+\:\d+')
for i in attacks['sentence']:
    time=time_regex.findall(i)
    attack_time.append(time)

In [446]:
attacks['Time']=attack_time

C:\Users\aleja\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [447]:
hashtag_regex=re.compile(r'[A-Z][a-z]+')
hashtag=[]
for i in attacks['hashtags']:
    i=str(i)
    hashs=hashtag_regex.findall(i)
    hashtag.append(hashs)


In [448]:
attacks['hashtags']=hashtag

C:\Users\aleja\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [449]:
attacks.head()

,date,hashtags,length,sentence,user,Check_Schemes,Time
4,2020-10-30 07:55:19,[Azerbaijan],23,Earlier today Azerbaijan’i Armed Forces launch...,ArmenianUnified,1,[]
13,2020-10-29 21:13:39,[Azerbaijan],43,The sounds of explosions in Stepanakert at the...,ArmenianUnified,1,[]
24,2020-10-29 12:34:58,"[Azerbaijan, Artsakh, Nagorno, Karabakh]",14,Azerbaijan continues shelling Shushi &amp; oth...,ArmenianUnified,1,[]
27,2020-10-29 10:44:46,"[Artsakh, Nagorno, Karabakh]",25,Armenian Unified Infocenter shares the images ...,ArmenianUnified,1,[]
41,2020-10-29 08:05:51,[Azerbaijan],19,The Azerbaijan|i forces have struck Stepanaker...,ArmenianUnified,1,[]


In [450]:
attacks

,date,hashtags,length,sentence,user,Check_Schemes,Time
4,2020-10-30 07:55:19,[Azerbaijan],23,Earlier today Azerbaijan’i Armed Forces launch...,ArmenianUnified,1,[]
13,2020-10-29 21:13:39,[Azerbaijan],43,The sounds of explosions in Stepanakert at the...,ArmenianUnified,1,[]
24,2020-10-29 12:34:58,"[Azerbaijan, Artsakh, Nagorno, Karabakh]",14,Azerbaijan continues shelling Shushi &amp; oth...,ArmenianUnified,1,[]
27,2020-10-29 10:44:46,"[Artsakh, Nagorno, Karabakh]",25,Armenian Unified Infocenter shares the images ...,ArmenianUnified,1,[]
41,2020-10-29 08:05:51,[Azerbaijan],19,The Azerbaijan|i forces have struck Stepanaker...,ArmenianUnified,1,[]
43,2020-10-29 08:05:51,[Azerbaijan],12,"There are ongoing missile attacks in Askeran,...",ArmenianUnified,1,[]
46,2020-10-29 07:35:52,[Azerbaijan],18,Adversary forces are now using Smerch volley f...,ArmenianUnified,1,[]
48,2020-10-29 05:54:11,[],15,RT ShStepanyan: Azerbaijani forces hit Artsakh...,ArmenianUnified,1,[]
55,2020-10-29 05:24:30,"[Artsakh, Azerbaijan, Artsakh]",14,"Specifically, the Azeri forces shelled Stepa...",ArmenianUnified,1,[]
56,2020-10-28 23:13:19,[Artsakh],12,Azerbaijan fired 5 missiles at the capital of ...,ArmenianUnified,1,[]


In [451]:
import os
import numpy as np
mainpath=os.getcwd()
fullpath=os.path.join(mainpath,'attacks')
attacks.to_csv(fullpath+".csv")

In [452]:
attacks.iloc[1,3]

'The sounds of explosions in Stepanakert at the moment are the sounds of explosions of ammunition destroyed as a result of targeted strikes by Karabakh_MoD units in the area of \u200b\u200boperation of Azerbaijan|i subversive groups The operation to destroy the subversive groups continues'

In [509]:

weapon=[]
for i in attacks['sentence']:
        weapon.append(re.compile(r'Smerch|smerch|missile|UAV|Bayraktar|drone|bayraktar|SU 25?').findall(i))
        
attacks['weapon']=weapon

C:\Users\aleja\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [510]:
attacks

,date,hashtags,length,sentence,user,Check_Schemes,Time,weapon,target
4,2020-10-30 07:55:19,[Azerbaijan],23,Earlier today Azerbaijan’i Armed Forces launch...,ArmenianUnified,1,[],[],"[Earlier, Azerbaijan, Armed, Forces, Northern]"
13,2020-10-29 21:13:39,[Azerbaijan],43,The sounds of explosions in Stepanakert at the...,ArmenianUnified,1,[],[],"[The, Stepanakert, Karabakh, Mo, Azerbaijan, The]"
24,2020-10-29 12:34:58,"[Azerbaijan, Artsakh, Nagorno, Karabakh]",14,Azerbaijan continues shelling Shushi &amp; oth...,ArmenianUnified,1,[],[],"[Azerbaijan, Shushi, Artsakh, Nagorno, Karabak..."
27,2020-10-29 10:44:46,"[Artsakh, Nagorno, Karabakh]",25,Armenian Unified Infocenter shares the images ...,ArmenianUnified,1,[],[],"[Armenian, Unified, Infocenter, Shushi, Azerba..."
41,2020-10-29 08:05:51,[Azerbaijan],19,The Azerbaijan|i forces have struck Stepanaker...,ArmenianUnified,1,[],"[Smerch, missile]","[The, Azerbaijan, Stepanakert, Smerch]"
43,2020-10-29 08:05:51,[Azerbaijan],12,"There are ongoing missile attacks in Askeran,...",ArmenianUnified,1,[],[missile],"[There, Askeran, Martuni, Martakert]"
46,2020-10-29 07:35:52,[Azerbaijan],18,Adversary forces are now using Smerch volley f...,ArmenianUnified,1,[],[Smerch],"[Adversary, Smerch, Stepanakert, Shushi, Marta..."
48,2020-10-29 05:54:11,[],15,RT ShStepanyan: Azerbaijani forces hit Artsakh...,ArmenianUnified,1,[],[Smerch],"[Sh, Stepanyan, Azerbaijani, Artsakh, Stepanak..."
55,2020-10-29 05:24:30,"[Artsakh, Azerbaijan, Artsakh]",14,"Specifically, the Azeri forces shelled Stepa...",ArmenianUnified,1,[],[Smerch],"[Specifically, Azeri, Stepanakert, Shushi, Sme..."
56,2020-10-28 23:13:19,[Artsakh],12,Azerbaijan fired 5 missiles at the capital of ...,ArmenianUnified,1,[],[missile],"[Azerbaijan, Artsakh]"


In [499]:
attacks.iloc[4][3]

'The Azerbaijan|i forces have struck Stepanakert with Smerch missiles during an air raid that lasted most than one hour'

In [513]:
target=[]
for i in attacks['sentence']:
        target.append(re.compile(r'Chambarak| Chinar| Nerkin| Karmiraghbyur|Aygepar| Chapni| Yeraskh|Khndzorut|Stepanakert|Shushi').findall(i))
        
attacks['target']=target

C:\Users\aleja\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [514]:
attacks['target']

4                                             []
13                                 [Stepanakert]
24                                      [Shushi]
27                                      [Shushi]
41                                 [Stepanakert]
43                                            []
46                         [Stepanakert, Shushi]
48                                 [Stepanakert]
55                         [Stepanakert, Shushi]
56                                            []
71                                      [Shushi]
78                                      [Shushi]
80                                 [Stepanakert]
82                                      [Shushi]
86                                 [Stepanakert]
89                                 [Stepanakert]
91                                            []
94                                      [Shushi]
96                                            []
131                                           []
149                 

In [515]:
attacks

,date,hashtags,length,sentence,user,Check_Schemes,Time,weapon,target
4,2020-10-30 07:55:19,[Azerbaijan],23,Earlier today Azerbaijan’i Armed Forces launch...,ArmenianUnified,1,[],[],[]
13,2020-10-29 21:13:39,[Azerbaijan],43,The sounds of explosions in Stepanakert at the...,ArmenianUnified,1,[],[],[Stepanakert]
24,2020-10-29 12:34:58,"[Azerbaijan, Artsakh, Nagorno, Karabakh]",14,Azerbaijan continues shelling Shushi &amp; oth...,ArmenianUnified,1,[],[],[Shushi]
27,2020-10-29 10:44:46,"[Artsakh, Nagorno, Karabakh]",25,Armenian Unified Infocenter shares the images ...,ArmenianUnified,1,[],[],[Shushi]
41,2020-10-29 08:05:51,[Azerbaijan],19,The Azerbaijan|i forces have struck Stepanaker...,ArmenianUnified,1,[],"[Smerch, missile]",[Stepanakert]
43,2020-10-29 08:05:51,[Azerbaijan],12,"There are ongoing missile attacks in Askeran,...",ArmenianUnified,1,[],[missile],[]
46,2020-10-29 07:35:52,[Azerbaijan],18,Adversary forces are now using Smerch volley f...,ArmenianUnified,1,[],[Smerch],"[Stepanakert, Shushi]"
48,2020-10-29 05:54:11,[],15,RT ShStepanyan: Azerbaijani forces hit Artsakh...,ArmenianUnified,1,[],[Smerch],[Stepanakert]
55,2020-10-29 05:24:30,"[Artsakh, Azerbaijan, Artsakh]",14,"Specifically, the Azeri forces shelled Stepa...",ArmenianUnified,1,[],[Smerch],"[Stepanakert, Shushi]"
56,2020-10-28 23:13:19,[Artsakh],12,Azerbaijan fired 5 missiles at the capital of ...,ArmenianUnified,1,[],[missile],[]


In [516]:
import os
import numpy as np
mainpath=os.getcwd()
fullpath=os.path.join(mainpath,'attacks')
attacks.to_csv(fullpath+".csv")

In [524]:
df4=df3.loc[df3['Check_Schemes']==0]

In [532]:
df4.iloc[:][1]

KeyError: 1